# Python names and variables

The Python system for naming and referencing variables can appear quite confusing at first, for those coming from a different background like *C/C++*. 

In this notebook:
- assignment operations.
- mutable and non-mutable variables.
- argument passing.

### References:
- Ned Batchelder's blog [post](https://nedbatchelder.com/text/names.html)
- Python [official documentation on assignment statements](https://docs.python.org/3/reference/simple_stmts.html#assignment-statements). 
- Dibya Chakravorty's blog [post](https://medium.com/broken-window/many-names-one-memory-address-122f78734cb6) on assignments.

## Mutable and immutable variables

The following snippet:

In [11]:
x = 1
y = x
x = 2
y

1

shows that updating `x` has no effect on `y` despite `y = x`. We get a better insight of what's happening by tracking the memory locations of each variable during execution:

In [47]:
x = 1
print('x=1, id x:', id(x))
y = x
print('y=x, id x:', id(x), 'id y:', id(y))
x = 2
print('x=2, id x:', id(x), 'id y:', id(y))

x=1, id x: 4417841232
y=x, id x: 4417841232 id y: 4417841232
x=2, id x: 4417841264 id y: 4417841232


Hence, the assignment `y=x` sets indeed the two names (`x` and `y`) to refer to the same location of memory. However, when `x=2` is executed, the name `x` is rebound to a different value. Part of the reason is that numbers are **immutable**, *e.g.*, their value cannot be changed. This means that the memory location storing `1` cannot be updated, and thus, when executing `x=2`, Python allocates the value `2` in a new location in memory, then rebinds name `x` to the new value. Numbers, strings and tuples are common examples of immutable variables. 

In [48]:
x = [1, 2, 3]
y = x
x.append(4)
y

[1, 2, 3, 4]

In this case, `x.append(4)` does not reassign name `x` to a new location (as one can easily check with `id()`), but merely alters its value. In other words, it performes a change *in-place*. Variables whose value can be updated are called **mutable**. Lists, dicts and user-defined objects are example of mutable variables. Clearly, we can also rebind the name of a mutable variable to a different values using assignments. For example:

In [49]:
x = [1, 2, 3]
y = x
x = x + [4]
y

[1, 2, 3]

Here, the RHS of the assignment operation creates a new object in memory and then rebinds the name `x` to it. The rule of thumb is that assignments rebind whereas methods perform in-place changes, although this is not entirely true. A notable exception is slicing:

In [36]:
x = [1, 2, 3]
x[0:2] = [1, 1]
x

[1, 1, 3]

What's the advantage of introducing immutable types in the language? The answer is performance. Consider the following snippet, and recall that strings are immutable:

In [37]:
string_1 = 'hello world'
string_2 = 'hello world'
print(id(string_1) == id(string_2))
string_3 = 'hello'
string_4 = 'hello'
print(id(string_3) == id(string_4))

False
True


Here, Python saves memory by using the same value for the names `string_3` and `string_4`. Understanding when Python decides to *intern* variables is non-trivial (in this example, the key is the white space) and should remain transparent to user. To prevent disasters, Python makes this sort of decision only on immutable variables, and the programmer is protected not being able to mutate their values.

## Argument passing
In Python, function arguments are always passed by reference (not by value), despite the argument being mutable or immutable. For example:

In [43]:
def fun(x):
    print(id(x))

y = [1, 2, 3]
print(id(y))
fun(y)

y = 1
print(id(y))
fun(y)

4452145864
4452145864
4417841232
4417841232


Argument passing is a type of assignment. This means that Python executes something analogous to `x = y`, when `y` is passed to `fun`.
Because argument passing occurs by reference, function computation can alter the values of the calling variables:

In [59]:
x = [1, 2]

def fun(y):
    y.append(3)
    y = y + [4]
    print('this is y:', y)
    
fun(x)
print('this is x:', x)

this is y: [1, 2, 3, 4]
this is x: [1, 2, 3]


Here's what happens:
- Line 1: `x` is assigned to a list (which is mutable).
- Line 3: `y` is assigned to same list referenced by `x`. 
- Line 4: `y.append(3)` performs an in-place change to the value. Both `x` and `y` are updated because they refer to the same value.
- Line 5: `y = y + [4]` creates a new list and rebinds `y` to that list. `x` remains unchanged.


It is also possible to emulate the pass-by-value feature of C language. The most Python way is probably the following (see topic on [StackExchange](https://stackoverflow.com/questions/845110/emulating-pass-by-value-behaviour-in-python)):

In [46]:
x = [1, 2]

def fun(y):
    y = y.copy()
    y.append(3)

fun(x)
x

[1, 2]

`copy()` can be used on any mutable object and returns a different object with the same value. Note that we would not need to copy an immutable object because by definition we cannot update its value.